In [1]:
from google.colab import drive
drive.mount('/mydrive')

Drive already mounted at /mydrive; to attempt to forcibly remount, call drive.mount("/mydrive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

In [3]:
tweetsdf = pd.read_csv('/mydrive/MyDrive/CMPE255/tweets.csv')

In [5]:
tweetsdf.head()

,content,class
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from sklearn import linear_model
import csv
import string
import io  
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
tweetsdf['content'] = tweetsdf['content'].apply(
    lambda x: ' '.join([word for word in (str(x)).split() if word not in (stop)]))
tweetsdf['content'] = tweetsdf.content.str.replace('[^a-zA-Z\s]', '')

In [8]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [9]:
labels = tweetsdf.pop('class')

In [10]:
X_train, X_test, y_train, y_test = train_test_split(tweetsdf,labels)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train['content'])

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2391748, 1956739)

In [13]:
X_new_counts = count_vect.transform(X_test['content'])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [14]:
# find number of components to keep
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=16)
tsvd.fit(X_train_counts)
X_test_svd = tsvd.transform(X_new_tfidf)
X_train_svd = tsvd.transform(X_train_tfidf)

# find explained variance 
#tsvd_var = tsvd.explained_variance_ratio_

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [19]:
clfs = []
clfs.append(LogisticRegression(max_iter=10000))
clfs.append(GaussianNB())
clfs.append(SGDClassifier(loss='modified_huber', shuffle=True, random_state=101, max_iter=100000))
clfs.append(KNeighborsClassifier(n_neighbors=4))
clfs.append(DecisionTreeClassifier(max_depth=10, random_state=101, max_features=None, min_samples_leaf=2))
clfs.append(RandomForestClassifier(n_estimators=70, oob_score=True, n_jobs=-1, random_state=101, max_features=None, min_samples_leaf=2))
clfs.append(SVC(kernel='linear', C=0.025, random_state=101))

In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
clf_preds = []
for clf in clfs:
    clf.fit(X_train_svd, y_train)
    clf_pred = clf.predict(X_test_svd)
    clf_preds.append(clf_pred)
    print(accuracy_score(y_test, clf_pred))

0.7857422389463782
0.6640627155848229
0.7738287864534337
0.8497949200376294
0.8169720915647538
